In [2]:
import pandas as pd
import numpy as np
import os
import glob
import pickle
from collections import Counter
import re
import csv

import xml.etree.ElementTree as ET


In [ ]:
listofdata = [] # this is my list that contains every single node/tag in every xml file that I downloaded


def parse_xml(folder):

    for x in glob.glob(folder + '/*.xml'):
        with open(x, 'rt', encoding='ISO-8859-1') as infile:
            tree = ET.parse(infile)
            for node in tree.iter():
                if node.tag == 'rootTag':
                    listofdata.append('rootTag')
                elif node.tag == 'AwardEffectiveDate':
                    listofdata.append(node.text)
                elif node.tag == 'AwardExpirationDate':
                    listofdata.append(node.text)
                elif node.tag == 'AwardAmount':
                    listofdata.append(node.text)
                elif node.tag == 'LongName':
                    listofdata.append(node.text)
                elif node.tag == 'AbstractNarration':
                    listofdata.append(node.text)
                elif node.tag == 'MinAmdLetterDate':
                    listofdata.append(node.text)
                elif node.tag == 'MaxAmdLetterDate':
                    listofdata.append(node.text)
                elif node.tag == 'StartDate':
                    listofdata.append(node.text)
                elif node.tag == 'Name':
                    listofdata.append(node.text)
                elif node.tag == 'StateName':
                    listofdata.append(node.text)
                else:
                    pass
                
            
    return listofdata


In [ ]:
dir_list = ['2013_NSF', '2014_NSF', '2015_NSF', '2016_NSF', '2017_NSF', '2018_NSF']

for d in dir_list:
    extract_xml(d)


In [ ]:
with open('listofdata.pickle', 'wb') as f:
    pickle.dump(listofdata, f)

In [ ]:
listofdata=pd.read_pickle('listofdata.pickle')

In [ ]:
# this step is to separate the list into list of lists, with each list belonging to one single xml file
roottag = re.compile(r'rootTag')
rtlist=[]
for i,v in enumerate(listofdata):
    if re.match(roottag, str(v)):
        rtlist.append(i)
    

In [ ]:
listofrecords = []
ind = 1
for r in rtlist:
    single_record = listofdata[r:rtlist[ind+1]]
    listofrecords.append(single_record)
    if ind < len(rtlist)-2:
        ind += 1
    else:
        break

In [ ]:
with open('listofrecords.pickle', 'wb') as f:
    pickle.dump(listofrecords, f)

In [ ]:
listofrecords = pd.read_pickle('listofrecords.pickle')

In [ ]:
df = pd.DataFrame(listofrecords)


In [ ]:
df.head()

### Now to convert timestamp columns into pandas datetime objects

In [ ]:
df['AwardEffdt']= pd.to_datetime(df[1])

In [ ]:
df['AwardExp_dt']= pd.to_datetime(df[2])

In [ ]:
df['Award_amt']= df[3].astype(int)

In [ ]:
df['Min_amend_dt']= pd.to_datetime(df[7])

In [ ]:
df['Max_amend_dt']= pd.to_datetime(df[8])

In [ ]:
df['Grant_length']=df['AwardExp_dt']-df['AwardEffdt']

In [ ]:
df['Amend_length'] = df['Max_amend_dt']-df['Min_amend_dt']

### Now I need to convert the 'grant length' and 'Amended days' column into the number of days 

In [ ]:
gl_days=[]
for i,x in enumerate(df['Grant_length']):
    gl_days.append(df['Grant_length'][i].days)

In [ ]:
df['Gl_days']= gl_days

In [ ]:
df['Gl_days'].dtype

In [ ]:
am_days = []
for i,x in enumerate(df['Amend_length']):
    am_days.append(df['Amend_length'][i].days)

In [ ]:
df['Am_days']=am_days

### Count the number of words in the abstract

In [ ]:
abs_words = []
for x in range(0, len(df)):
    try:
        abs_words.append(len(df[5][x]))
     #   abs_words.append(len(award_data2['Abstract'][x]))
    except:
        abs_words.append('0')
     #   pass

In [ ]:
df['Abstract_length']=abs_words

In [ ]:
df['Abstract_length']=df['Abstract_length'].astype(int)

In [ ]:
with open('df.pickle', 'wb') as f:
    pickle.dump(df, f)

In [ ]:
df = pd.read_pickle('df.pickle')

### Get dummy variable for the 'Directorate' column (Biology, Math, Physics, etc)

In [ ]:
Counter(df[4])

In [ ]:
Z = pd.get_dummies(df[4])
Z.head()

In [ ]:
df2=df.join(Z)

In [ ]:
df2.head()

In [ ]:
with open('df_dummy.pickle', 'wb') as f:
    pickle.dump(df2, f)

In [ ]:
df = pd.read_pickle('df_dummy.pickle')

### Add State data

In [ ]:
stdata = pd.read_csv('statelist.csv', sep=',')

In [ ]:
df2 = pd.merge(df,stdata, on='Index', sort=False, how='left')

In [ ]:
with open('df_dept_sts.pickle', 'wb') as f:
    pickle.dump(df2, f)

In [ ]:
df2 = pd.read_pickle('df_dept_sts.pickle')

### Add University/Institution data (had to clean this up because this field was spread out among 5 different columns)

In [ ]:
univdata = pd.read_csv('combined_univ.csv', sep=',')

In [ ]:
df3 = pd.merge(df2,univdata, on='Index', sort=False,how='left')

In [ ]:
with open('df_dept_sts_univ.pickle', 'wb') as f:
    pickle.dump(df3, f)

In [ ]:
df = pd.read_pickle('df_dept_sts_univ.pickle')

### Add University/Institution Ranking data

In [ ]:
rankings=pd.read_csv('inst_RD_ranking.csv', sep=',')

In [ ]:
df4 = pd.merge(df,rankings, on='Index', sort=False,how='left')

In [ ]:
df4 = df4.drop('Institution_y', axis=1)

In [ ]:
df4 = df4.drop([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,
                24,25,26,27,28,29,30,31,32,33,34,35,36], axis=1)

In [ ]:
with open('final_cleaned_data.pickle', 'wb') as f:
    pickle.dump(df4, f)

In [ ]:
df = pd.read_pickle('final_cleaned_data.pickle')

In [ ]:
with open('modelready_data.pickle', 'wb') as f:
    pickle.dump(df4, f)

In [4]:
df = pd.read_pickle('modelready_data.pickle')

In [5]:
df.columns

Index(['AwardEffdt', 'AwardExp_dt', 'Award_amt', 'Min_amend_dt',
       'Max_amend_dt', 'Grant_length', 'Amend_length', 'Gl_days', 'Am_days',
       'Abstract_length', 'Direct For Biological Sciences',
       'Direct For Computer & Info Scie & Enginr',
       'Direct For Education and Human Resources',
       'Direct For Mathematical & Physical Scien',
       'Direct For Social, Behav & Economic Scie',
       'Directorate For Engineering', 'Directorate For Geosciences',
       'Directorate for Geosciences', 'National Coordination Office',
       'Natl Nanotechnology Coordinating Office',
       'Office Of Information & Resource Mgmt', 'Office Of Polar Programs',
       'Office Of The Director',
       'Office of Budget, Finance, & Award Management', 'Index', 'State',
       'Institution_x', '2016_RD_exp_rank', '2016_RD_perc',
       'Rank_gradstudents_x', 'Rank_doctorates', 'Rank_gradstudents_y'],
      dtype='object')

In [4]:
df=df.drop('Index', axis=1)

In [ ]:
df.columns

### Get dummy variables for University/Institution

In [20]:
U = pd.get_dummies(df['Institution_x'])
U = U.drop(['Direct For Biological Sciences','Direct For Computer & Info Scie & Enginr',
       'Direct For Education and Human Resources','Direct For Mathematical & Physical Scien',
       'Direct For Social, Behav & Economic Scie', 'Directorate For Engineering', 'Directorate For Geosciences',
       'Office Of Information & Resource Mgmt', 'Office Of Polar Programs','Office Of The Director',
            'Alaska Pacific University','University of Alaska Anchorage Campus',
            'University of Alaska Fairbanks Campus',  'University of Alaska Southeast Juneau Campus'], axis=1)

In [21]:
U.head()

,18F GSA,"2B Technologies, Inc",2W iTech LLC,"3D Biotek, LLC","3DEO, Inc.","3Derm Systems, Inc.","3I Diagnostics, Inc.",4 D Technology Corporation,4-Web Spine Inc.,490 BioTech Inc,...,schultzhaus Zachary,selfarray,shark wheel,"unspun, Inc.",vonSternberg Nicholas C,whitaker melissa r,wondervisions,yearONE LLC,"zGlue, Inc.",zeroK NanoTech Corporation
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
df2=df.join(U)

### Get dummy variables for State

In [6]:
S = pd.get_dummies(df['State'])
S.head()

,Alabama,Alaska,Alaska Pacific University,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,District of Columbia,...,University of Alaska Fairbanks Campus,University of Alaska Southeast Juneau Campus,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
df3=df2.join(S)

In [ ]:
with open('finalized_data.pickle', 'wb') as f:
    pickle.dump(df3, f)